In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import asyncio
import json

In [4]:
def get_position(url):
    positions = []
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 YaBrowser/24.7.0.0 Safari/537.36'}
    try:
        res = requests.get(url, headers=headers, timeout=120)
    except requests.exceptions.ConnectionError as e:
        res = "No response"
    comm = re.compile("<!--|-->")
    if res != 'No response':
        soup = BeautifulSoup(comm.sub("", res.text), "lxml")
        x = soup.findAll('body')[0]
        pos = x.find_all('a', {'class': 'sr_preset', 'data-hide': "#all_similar .section_heading, #all_similar .topscroll_div"})
        for i in pos:
            p = i.text.strip().encode().decode("utf-8")
            if p == 'Att Mid / Wingers':
                positions.append('AM')
            elif p == 'Midfielders':
                positions.append('CM')
            elif p == 'Center Backs':
                positions.append('CB')
            elif p == 'Fullbacks':
                positions.append('FB')
            elif p == 'Goalkeepers':
                positions.append('GK')
            else:
                positions.append('FW')
    else:
        positions = [res]
    return ', '.join(positions)

In [5]:
df = pd.read_csv('T524_Outfield.csv')
links = df['position'].to_list()

In [ ]:
number = 0
positions = []

In [ ]:
while number < len(links):
    for i in range(number, number + 19):
        if i < len(links):
            p = get_position(links[i])
            print(p)
            positions.append(p)
    number += 19
    await asyncio.sleep(60)

In [36]:
with open("positions.json", "w", encoding="utf-8") as file:
    json.dump(positions, file)

In [37]:
f = open('positions.json')
positions = json.load(f)

In [41]:
df['position']= positions

In [43]:
df.to_csv('T524_Outfield.csv')